In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the VGG16 model, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model to retain pretrained weights
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for our specific task
x = base_model.output
x = Flatten()(x)  # Flatten the output from the convolutional layers
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dense(128, activation='relu')(x)  # Additional fully connected layer
x = Dense(4, activation='softmax')(x)  # Output layer for 4 classes

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model architecture
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 27,626,436 (105.39 MB)

 Trainable params: 12,911,748 (49.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/plant-pathology-2020-fgvc7/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_df, val_df = train_test_split(train, test_size=0.2, stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']], random_state=42)

In [ ]:
pip install tensorflowjs

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflowjs as tfjs
# Assuming `train` is a DataFrame with your training data and labels
X = train['image_id'].values
y = train[['healthy', 'multiple_diseases', 'rust', 'scab']].values

# Number of splits/folds
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

fold = 1
base_dir = '/content/drive/MyDrive/plant-pathology-2020-fgvc7'

for train_index, val_index in kf.split(X):
    print(f'Fold {fold}')

    train_df = train.iloc[train_index]
    val_df = train.iloc[val_index]

    # Create directories for this fold
    fold_train_dir = os.path.join(base_dir, f'train_fold_{fold}')
    fold_val_dir = os.path.join(base_dir, f'val_fold_{fold}')
    os.makedirs(fold_train_dir, exist_ok=True)
    os.makedirs(fold_val_dir, exist_ok=True)

    for cls in ['healthy', 'multiple_diseases', 'rust', 'scab']:
        os.makedirs(os.path.join(fold_train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(fold_val_dir, cls), exist_ok=True)

    img_dir = os.path.join(base_dir, 'images')

    # Move images to corresponding directories for training set
    for index, row in train_df.iterrows():
        img_id = row['image_id']
        img_path = f'{img_dir}/{img_id}.jpg'  # Adjust image format as necessary
        for cls in ['healthy', 'multiple_diseases', 'rust', 'scab']:
            if row[cls]:  # Assuming class columns contain binary indicators (1/0)
                shutil.copy(img_path, os.path.join(fold_train_dir, cls))

    # Move images to corresponding directories for validation set
    for index, row in val_df.iterrows():
        img_id = row['image_id']
        img_path = f'{img_dir}/{img_id}.jpg'  # Adjust image format as necessary
        for cls in ['healthy', 'multiple_diseases', 'rust', 'scab']:
            if row[cls]:  # Assuming class columns contain binary indicators (1/0)
                shutil.copy(img_path, os.path.join(fold_val_dir, cls))

    # Set up data generators with batch size of 16
    batch_size = 16
    train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, rotation_range=15)
    validation_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        fold_train_dir,
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical'
    )
    validation_generator = validation_datagen.flow_from_directory(
        fold_val_dir,  # Path to validation data for this fold
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Train the model
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator
    )
        # Specify the path to save the model in the native Keras format
    model_save_path = os.path.join(base_dir, f'model_fold_{fold}.keras')

    # Save the model in the native Keras format
    model.save(model_save_path)

    # Directory to save the model in TensorFlow.js format
    tfjs_save_path = os.path.join(base_dir, f'model_fold_{fold}_tfjs')

    # Convert and save the model in TensorFlow.js format
    tfjs.converters.save_keras_model(model, tfjs_save_path)

    # Optionally, save the model and history for each fold
    # Add any other code to save history or results

    fold += 1


Fold 1
Found 1456 images belonging to 4 classes.
Found 365 images belonging to 4 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


91/91 ━━━━━━━━━━━━━━━━━━━━ 54s 561ms/step - accuracy: 0.9208 - loss: 0.2222 - val_accuracy: 0.9562 - val_loss: 0.1645
Epoch 2/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 80s 542ms/step - accuracy: 0.9549 - loss: 0.1457 - val_accuracy: 0.9178 - val_loss: 0.2249
Epoch 3/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 82s 542ms/step - accuracy: 0.9692 - loss: 0.1231 - val_accuracy: 0.9370 - val_loss: 0.1969
Epoch 4/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 78s 502ms/step - accuracy: 0.9650 - loss: 0.1130 - val_accuracy: 0.9370 - val_loss: 0.2009
Epoch 5/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 48s 491ms/step - accuracy: 0.9433 - loss: 0.1496 - val_accuracy: 0.9205 - val_loss: 0.2461
Epoch 6/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 84s 520ms/step - accuracy: 0.9749 - loss: 0.0934 - val_accuracy: 0.9507 - val_loss: 0.1781
Epoch 7/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 83s 533ms/step - accuracy: 0.9751 - loss: 0.1029 - val_accuracy: 0.8740 - val_loss: 0.2859
Epoch 8/10
91/91 ━━━━━━━━━━━━━━━━━━━━ 84s 563ms/step - accuracy: 0.9577 - loss: 0.1175 - val_accuracy: 0.926

failed to lookup keras version from the file,
    this is likely a weight only file
Fold 2
Found 1457 images belonging to 4 classes.
Found 364 images belonging to 4 classes.
Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 55s 570ms/step - accuracy: 0.9359 - loss: 0.1906 - val_accuracy: 0.9698 - val_loss: 0.0811
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 56s 574ms/step - accuracy: 0.9613 - loss: 0.1187 - val_accuracy: 0.9780 - val_loss: 0.0782
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 78s 531ms/step - accuracy: 0.9653 - loss: 0.0973 - val_accuracy: 0.9890 - val_loss: 0.0596
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 514ms/step - accuracy: 0.9778 - loss: 0.0790 - val_accuracy: 0.9451 - val_loss: 0.1558
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 51s 528ms/step - accuracy: 0.9814 - loss: 0.0652 - val_accuracy: 0.9396 - val_loss: 0.1442
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 509ms/step - accuracy: 0.9650 - loss: 0.1136 - val_accuracy: 0.9725 - val_loss: 0.0909
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 53s 546ms/step

failed to lookup keras version from the file,
    this is likely a weight only file
Fold 3


KeyboardInterrupt: 

In [8]:
from google.colab import files
files.download('/content/drive/MyDrive/plant-pathology-2020-fgvc7/model_fold_2.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from tensorflow.keras.models import load_model

In [6]:
pip install tensorflowjs

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully unin

In [5]:
import tensorflowjs as tfjs

ModuleNotFoundError: No module named 'tensorflowjs'

In [4]:
model = load_model('/content/model_fold_2.keras')

# Directory where the TensorFlow.js model will be saved
tfjs_output_dir = '/content/'

# Convert and save the model in TensorFlow.js format
tfjs.converters.save_keras_model(model, tfjs_output_dir)

NameError: name 'tfjs' is not defined